Morgan Turville-Heitz
CS 760
10/18/2023

In [1]:

import pandas as pd
import numpy as np
import os
import ipykernel

Defining the Doc class, where counts/vectors are stored for each document.

In [14]:
class Doc():

    def __init__(self, raw, fn):
        self.fn = fn
        self.vector = self.vectorize(raw)
        self.char_counts = self.count(self.vector)
        self.label = fn[0]

    def vectorize(self, data):
        ### Stripping newlines/special characters
        filtered = ''.join([char for char in data if char.isalpha() or char == ' '])
        return list(filtered)

    def count(self, vector):
        count = {char: 0 for char in 'abcdefghijklmnopqrstuvwxyz '}
        for char in vector:
            count[char]+=1
        return count

Loading files, creating the Doc class for each file.

In [15]:
direc = r"C:\Users\Meau\Documents\GRAD SCHOOL\CS 760\Hw4\languageID"
files = os.listdir(direc)
for txtfile in files:
    path = os.path.join(direc, txtfile)
    with open(path, 'r') as file:
        raw = file.read()
    Doc(raw, str(txtfile))